<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-05/NLP_and_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

In [ ]:
def softmax(scores):
  return np.exp(scores) / np.sum(np.exp(scores), axis = 1)

def scaled_dot_product_attention(queries, keys, values):
  scores = np.matmul(queries, keys.T) / np.sqrt(np.shape(keys)[-1])
  return softmax(scores)


# class DotProductAttention(nn.Layer):
#   def __init__(self):
#     super().__init__()

#   def call(self, queries, keys, values):
